# The Springfield Identity - OPTIMIZED Training Pipeline
## High-Performance CNN Training with Detailed Progress Tracking

### Optimizations:
- **Full CPU Utilization**: Maximized num_workers and persistent workers
- **Efficient Data Loading**: Prefetch factor and pin memory
- **Detailed Progress Bars**: Multi-level tracking with real-time metrics
- **Early Stopping**: Prevent overfitting
- **Gradient Accumulation**: Effective larger batch size
- **Learning Rate Scheduling**: Adaptive learning rate

In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import json
from collections import defaultdict
import time
from datetime import datetime
import multiprocessing
import platform

if platform.system() == 'Windows':
    NUM_WORKERS = 0
    print("   Windows detected: Using single-threaded data loading (num_workers=0)")
    print("   This avoids slow multiprocessing spawn overhead on Windows")
else:
    NUM_WORKERS = min(multiprocessing.cpu_count(), 8)
print(f"CPU Cores Detected: {multiprocessing.cpu_count()}")
print(f"Using {NUM_WORKERS} workers for data loading")

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
SEED = 42
set_seed(SEED)

torch.set_num_threads(multiprocessing.cpu_count())

print("\n" + "="*80)
print("THE SPRINGFIELD IDENTITY - OPTIMIZED CNN TRAINING")
print("="*80)
print(f"Random Seed: {SEED}")
print(f"PyTorch Version: {torch.__version__}")
print(f"PyTorch Threads: {torch.get_num_threads()}")
print(f"Training Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

⚠️  Windows detected: Using single-threaded data loading (num_workers=0)
   This avoids slow multiprocessing spawn overhead on Windows
CPU Cores Detected: 32
Using 0 workers for data loading

THE SPRINGFIELD IDENTITY - OPTIMIZED CNN TRAINING
Random Seed: 42
PyTorch Version: 2.9.1+cpu
PyTorch Threads: 32
Training Started: 2025-12-10 23:35:03


C:\Users\andrua\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CONFIG = {
    'data_dir': 'characters_train',
    'img_size': 128,               
    'batch_size': 64,  # Increased for better CPU utilization
    'epochs': 25,                   
    'lr': 0.001,
    'weight_decay': 1e-4,
    'early_stopping_patience': 5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'model_save_path': 'model.pth',
    'num_workers': NUM_WORKERS,
    'prefetch_factor': 4,
    'persistent_workers': True if NUM_WORKERS > 0 else False,
    'grad_clip': 1.0,
    'grad_accum_steps': 1
}

print("\n" + "="*80)
print("OPTIMIZED CONFIGURATION")
print("="*80)
for key, value in CONFIG.items():
    print(f"{key:.<35} {value}")
print("="*80)
print(f"\nEffective Batch Size: {CONFIG['batch_size'] * CONFIG['grad_accum_steps']}")
print(f"Total Data Loading Workers: {CONFIG['num_workers']}")
print(f"Prefetch Buffer per Worker: {CONFIG['prefetch_factor']} batches")


OPTIMIZED CONFIGURATION
data_dir........................... characters_train
img_size........................... 128
batch_size......................... 64
epochs............................. 25
lr................................. 0.001
weight_decay....................... 0.0001
early_stopping_patience............ 5
device............................. cpu
model_save_path.................... model.pth
num_workers........................ 0
prefetch_factor.................... 4
persistent_workers................. False
grad_clip.......................... 1.0
grad_accum_steps................... 1

Effective Batch Size: 64
Total Data Loading Workers: 0
Prefetch Buffer per Worker: 4 batches


In [3]:
print("\n" + "="*80)
print("DATA LOADING")
print("="*80)

train_transforms = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size']), interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((CONFIG['img_size'], CONFIG['img_size']), interpolation=transforms.InterpolationMode.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("\n Loading dataset...")
load_start = time.time()

try:
    full_dataset = datasets.ImageFolder(root=CONFIG['data_dir'])
    print(f"✓ Dataset loaded in {time.time() - load_start:.2f}s")
except FileNotFoundError:
    print(f"✗ Error: '{CONFIG['data_dir']}' not found!")
    raise

class_names = full_dataset.classes
num_classes = len(class_names)
total_images = len(full_dataset)

print(f"\n Dataset Statistics:")
print(f"  Total Images............ {total_images:,}")
print(f"  Number of Classes....... {num_classes}")
print(f"  Classes: {', '.join(class_names[:5])}{'...' if num_classes > 5 else ''}")

print("\n Analyzing class distribution...")
class_counts = defaultdict(int)
for _, label in tqdm(full_dataset, desc="Scanning", unit="img", leave=False):
    class_counts[label] += 1

print(f"  Min samples/class....... {min(class_counts.values()):,}")
print(f"  Max samples/class....... {max(class_counts.values()):,}")
print(f"  Mean samples/class...... {np.mean(list(class_counts.values())):.1f}")
print(f"  Median samples/class.... {np.median(list(class_counts.values())):.1f}")

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(
    full_dataset, 
    [train_size, val_size], 
    generator=torch.Generator().manual_seed(SEED)
)

print(f"\n Data Split:")
print(f"  Train............. {train_size:,} ({train_size/total_images*100:.1f}%)")
print(f"  Validation........ {val_size:,} ({val_size/total_images*100:.1f}%)")

class TransformSubset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y
        
    def __len__(self):
        return len(self.subset)

train_data = TransformSubset(train_dataset, train_transforms)
val_data = TransformSubset(val_dataset, val_transforms)

print(f"\n⚡ Creating optimized DataLoaders...")

train_loader = DataLoader(
    train_data, 
    batch_size=CONFIG['batch_size'], 
    shuffle=True, 
    num_workers=CONFIG['num_workers'],
    pin_memory=True if CONFIG['device'] == 'cuda' else False,
    prefetch_factor=CONFIG['prefetch_factor'] if CONFIG['num_workers'] > 0 else None,
    persistent_workers=CONFIG['persistent_workers']
)

val_loader = DataLoader(
    val_data, 
    batch_size=CONFIG['batch_size'], 
    shuffle=False, 
    num_workers=CONFIG['num_workers'],
    pin_memory=True if CONFIG['device'] == 'cuda' else False,
    prefetch_factor=CONFIG['prefetch_factor'] if CONFIG['num_workers'] > 0 else None,
    persistent_workers=CONFIG['persistent_workers']
)

print(f"✓ DataLoaders ready")
print(f"  Batch size.............. {CONFIG['batch_size']}")
print(f"  Train batches........... {len(train_loader)}")
print(f"  Val batches............. {len(val_loader)}")
print(f"  Workers................. {CONFIG['num_workers']}")
print(f"  Prefetch factor......... {CONFIG['prefetch_factor']}")
print(f"  Persistent workers...... {CONFIG['persistent_workers']}")
print("="*80)


DATA LOADING

📁 Loading dataset...
✓ Dataset loaded in 0.02s

📊 Dataset Statistics:
  Total Images............ 16,764
  Number of Classes....... 42
  Classes: abraham_grampa_simpson, agnes_skinner, apu_nahasapeemapetilon, barney_gumble, bart_simpson...

🔍 Analyzing class distribution...


  Min samples/class....... 3
  Max samples/class....... 1,797
  Mean samples/class...... 399.1
  Median samples/class.... 124.0

✂️ Data Split:
  Train............. 13,411 (80.0%)
  Validation........ 3,353 (20.0%)

⚡ Creating optimized DataLoaders...
✓ DataLoaders ready
  Batch size.............. 64
  Train batches........... 210
  Val batches............. 53
  Workers................. 0
  Prefetch factor......... 4
  Persistent workers...... False


In [4]:
print("\n" + "="*80)
print("MODEL ARCHITECTURE")
print("="*80)

class SimpsonsCNN(nn.Module):
    """Optimized CNN for Simpsons Character Classification"""
    
    def __init__(self, num_classes):
        super(SimpsonsCNN, self).__init__()
        
        def conv_block(in_c, out_c, pool=True):
            layers = [
                nn.Conv2d(in_c, out_c, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(out_c),
                nn.ReLU(inplace=True)
            ]
            if pool:
                layers.append(nn.MaxPool2d(2))
            return nn.Sequential(*layers)
        
        self.block1 = conv_block(3, 32)
        self.block2 = conv_block(32, 64)
        self.block3 = conv_block(64, 128)
        self.block4 = conv_block(128, 256)
        
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.global_pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

print("\n🏗️ Initializing model...")
model = SimpsonsCNN(num_classes).to(CONFIG['device'])

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✓ Model initialized")
print(f"  Architecture............ SimpsonsCNN")
print(f"  Total parameters........ {total_params:,}")
print(f"  Trainable parameters.... {trainable_params:,}")
print(f"  Model size.............. {total_params * 4 / (1024**2):.2f} MB")
print(f"  Device.................. {CONFIG['device']}")
print("="*80)


MODEL ARCHITECTURE

🏗️ Initializing model...
✓ Model initialized
  Architecture............ SimpsonsCNN
  Total parameters........ 1,184,010
  Trainable parameters.... 1,184,010
  Model size.............. 4.52 MB
  Device.................. cpu


In [5]:
print("\n" + "="*80)
print("TRAINING SETUP")
print("="*80)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(
    model.parameters(), 
    lr=CONFIG['lr'], 
    weight_decay=CONFIG['weight_decay']
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max', 
    factor=0.5, 
    patience=2,
)

print(f"\n⚙️ Optimizer Configuration:")
print(f"  Loss function........... CrossEntropyLoss")
print(f"  Optimizer............... AdamW")
print(f"  Learning rate........... {CONFIG['lr']}")
print(f"  Weight decay............ {CONFIG['weight_decay']}")
print(f"  LR scheduler............ ReduceLROnPlateau")
print(f"  Gradient clipping....... {CONFIG['grad_clip']}")
print(f"  Gradient accum steps.... {CONFIG['grad_accum_steps']}")
print(f"  Early stopping.......... {CONFIG['early_stopping_patience']} epochs")
print("="*80)


TRAINING SETUP

⚙️ Optimizer Configuration:
  Loss function........... CrossEntropyLoss
  Optimizer............... AdamW
  Learning rate........... 0.001
  Weight decay............ 0.0001
  LR scheduler............ ReduceLROnPlateau
  Gradient clipping....... 1.0
  Gradient accum steps.... 1
  Early stopping.......... 5 epochs


In [6]:
def train_one_epoch(model, loader, optimizer, criterion, epoch, total_epochs, grad_accum_steps=1):
    """Train for one epoch with detailed progress tracking"""
    model.train()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    pbar = tqdm(
        enumerate(loader), 
        total=len(loader),
        desc=f"Epoch {epoch:2d}/{total_epochs} [TRAIN]",
        unit="batch",
        leave=False,
        bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}] {postfix}',
        colour='blue'
    )
    
    for batch_idx, (images, labels) in pbar:
        images, labels = images.to(CONFIG['device']), labels.to(CONFIG['device'])
        
        outputs = model(images)
        loss = criterion(outputs, labels) / grad_accum_steps
        
        loss.backward()
        
        if (batch_idx + 1) % grad_accum_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), CONFIG['grad_clip'])
            optimizer.step()
            optimizer.zero_grad()
        
        running_loss += loss.item() * grad_accum_steps
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        if (batch_idx + 1) % 5 == 0:
            current_loss = running_loss / (batch_idx + 1)
            current_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
            pbar.set_postfix({
                'loss': f'{current_loss:.4f}',
                'f1': f'{current_f1:.3f}',
                'lr': f'{optimizer.param_groups[0]["lr"]:.6f}'
            })
    
    pbar.close()
    
    epoch_loss = running_loss / len(loader)
    epoch_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    
    return epoch_loss, epoch_f1

def validate(model, loader, criterion, epoch, total_epochs):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    pbar = tqdm(
        enumerate(loader),
        total=len(loader),
        desc=f"Epoch {epoch:2d}/{total_epochs} [VALID]",
        unit="batch",
        leave=False,
        bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}] {postfix}',
        colour='green'
    )
    
    with torch.no_grad():
        for batch_idx, (images, labels) in pbar:
            images, labels = images.to(CONFIG['device']), labels.to(CONFIG['device'])
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            if (batch_idx + 1) % 5 == 0:
                current_loss = running_loss / (batch_idx + 1)
                current_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
                pbar.set_postfix({
                    'loss': f'{current_loss:.4f}',
                    'f1': f'{current_f1:.3f}'
                })
    
    pbar.close()
    
    epoch_loss = running_loss / len(loader)
    epoch_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    
    return epoch_loss, epoch_f1

print("\n✓ Training functions initialized with detailed progress tracking")


✓ Training functions initialized with detailed progress tracking


In [7]:
print("\n" + "="*80)
print("🚀 STARTING OPTIMIZED TRAINING")
print("="*80)

best_f1 = 0.0
best_epoch = 0
epochs_without_improvement = 0
training_history = {
    'train_loss': [],
    'train_f1': [],
    'val_loss': [],
    'val_f1': [],
    'learning_rates': [],
    'epoch_times': []
}

start_time = time.time()

epoch_pbar = tqdm(
    range(1, CONFIG['epochs'] + 1),
    desc="📊 Training Progress",
    unit="epoch",
    bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}] {postfix}',
    colour='cyan',
    position=0
)

print("\n" + "-"*80)
print(f"{'Epoch':^6} | {'Time':^7} | {'LR':^10} | {'Train Loss':^11} | {'Train F1':^9} | {'Val Loss':^11} | {'Val F1':^9} | {'Status':^15}")
print("-"*80)

for epoch in epoch_pbar:
    epoch_start_time = time.time()
    
    current_lr = optimizer.param_groups[0]['lr']
    
    train_loss, train_f1 = train_one_epoch(
        model, train_loader, optimizer, criterion, 
        epoch, CONFIG['epochs'], CONFIG['grad_accum_steps']
    )
    
    val_loss, val_f1 = validate(
        model, val_loader, criterion, epoch, CONFIG['epochs']
    )
    
    scheduler.step(val_f1)
    
    epoch_time = time.time() - epoch_start_time
    training_history['train_loss'].append(train_loss)
    training_history['train_f1'].append(train_f1)
    training_history['val_loss'].append(val_loss)
    training_history['val_f1'].append(val_f1)
    training_history['learning_rates'].append(current_lr)
    training_history['epoch_times'].append(epoch_time)
    
    status = ""
    if val_f1 > best_f1:
        improvement = val_f1 - best_f1
        best_f1 = val_f1
        best_epoch = epoch
        epochs_without_improvement = 0
        status = f"⭐ BEST (+{improvement:.4f})"
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_f1': best_f1,
            'class_names': class_names,
            'config': CONFIG,
            'training_history': training_history
        }
        torch.save(checkpoint, CONFIG['model_save_path'])
    else:
        epochs_without_improvement += 1
        status = f"({epochs_without_improvement}/{CONFIG['early_stopping_patience']})"
    
    print(f"{epoch:^6} | {epoch_time:>6.1f}s | {current_lr:>10.6f} | "
          f"{train_loss:>11.4f} | {train_f1:>9.4f} | "
          f"{val_loss:>11.4f} | {val_f1:>9.4f} | {status:^15}")
    
    epoch_pbar.set_postfix({
        'Best_F1': f'{best_f1:.4f}',
        'Val_F1': f'{val_f1:.4f}',
        'ETA': f'{np.mean(training_history["epoch_times"]) * (CONFIG["epochs"] - epoch) / 60:.1f}m'
    })
    
    if epochs_without_improvement >= CONFIG['early_stopping_patience']:
        print("-"*80)
        print(f"\n🛑 EARLY STOPPING: No improvement for {CONFIG['early_stopping_patience']} epochs")
        print(f"   Best F1: {best_f1:.4f} at epoch {best_epoch}")
        break

epoch_pbar.close()
print("-"*80)

total_time = time.time() - start_time
print("\n" + "="*80)
print("✅ TRAINING COMPLETE")
print("="*80)
print(f"\n⏱️ Time Statistics:")
print(f"  Total time.............. {total_time/60:.2f} minutes ({total_time:.0f}s)")
print(f"  Average per epoch....... {np.mean(training_history['epoch_times']):.2f}s")
print(f"  Fastest epoch........... {min(training_history['epoch_times']):.2f}s")
print(f"  Slowest epoch........... {max(training_history['epoch_times']):.2f}s")

print(f"\n📈 Performance:")
print(f"  Best Validation F1...... {best_f1:.4f} (Epoch {best_epoch})")
print(f"  Final Train F1.......... {training_history['train_f1'][-1]:.4f}")
print(f"  Final Val F1............ {training_history['val_f1'][-1]:.4f}")
print(f"  Initial Val F1.......... {training_history['val_f1'][0]:.4f}")
print(f"  Total Improvement....... {training_history['val_f1'][-1] - training_history['val_f1'][0]:+.4f}")

print(f"\n💾 Output Files:")
print(f"  Model checkpoint........ {CONFIG['model_save_path']}")

history_file = 'training_history.json'
with open(history_file, 'w') as f:
    json.dump(training_history, f, indent=4)
print(f"  Training history........ {history_file}")

print("\n" + "="*80)
print("🎉 SUCCESS! Model is ready for inference.")
print("="*80)


🚀 STARTING OPTIMIZED TRAINING


📊 Training Progress:   0%|          | 0/25 [00:00<?] 


--------------------------------------------------------------------------------
Epoch  |  Time   |     LR     | Train Loss  | Train F1  |  Val Loss   |  Val F1   |     Status     
--------------------------------------------------------------------------------



Epoch  1/25 [TRAIN]: 100%|██████████| 210/210 [02:27<00:00,  1.59batch/s] , loss=2.7872, f1=0.077, lr=0.001000
                                                                                                              
📊 Training Progress:   4%|▍         | 1/25 [02:43<1:05:32] , Best_F1=0.1367, Val_F1=0.1367, ETA=65.5m

  1    |  163.8s |   0.001000 |      2.7872 |    0.0767 |      2.3561 |    0.1367 | ⭐ BEST (+0.1367)


...

  23   |  151.7s |   0.001000 |      0.2617 |    0.7898 |      0.3080 |    0.7648 |      (1/5)     



Epoch 24/25 [TRAIN]: 100%|██████████| 210/210 [02:18<00:00,  1.64batch/s] , loss=0.2468, f1=0.784, lr=0.001000
                                                                                                              
📊 Training Progress:  96%|█████████▌| 24/25 [1:06:04<02:36] , Best_F1=0.7775, Val_F1=0.7775, ETA=2.8m

  24   |  153.9s |   0.001000 |      0.2468 |    0.7843 |      0.2719 |    0.7775 | ⭐ BEST (+0.0115)



Epoch 25/25 [TRAIN]: 100%|██████████| 210/210 [02:17<00:00,  1.77batch/s] , loss=0.2375, f1=0.802, lr=0.001000
                                                                                                              
📊 Training Progress: 100%|██████████| 25/25 [1:08:37<00:00] , Best_F1=0.7775, Val_F1=0.7056, ETA=0.0m

  25   |  152.7s |   0.001000 |      0.2375 |    0.8018 |      0.3170 |    0.7056 |      (1/5)     
--------------------------------------------------------------------------------

✅ TRAINING COMPLETE

⏱️ Time Statistics:
  Total time.............. 68.62 minutes (4117s)
  Average per epoch....... 164.68s
  Fastest epoch........... 149.67s
  Slowest epoch........... 224.04s

📈 Performance:
  Best Validation F1...... 0.7775 (Epoch 24)
  Final Train F1.......... 0.8018
  Final Val F1............ 0.7056
  Initial Val F1.......... 0.1367
  Total Improvement....... +0.5688

💾 Output Files:
  Model checkpoint........ model.pth
  Training history........ training_history.json

🎉 SUCCESS! Model is ready for inference.
